In [ ]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#SDXL で必要なモジュールのインストール
!pip install -U peft tensorflow-metadata diffusers transformers scikit-learn ftfy accelerate invisible_watermark safetensors controlnet-aux mediapipe timm GPUtil
#サーバとして利用するためのモジュールのインストール
!pip install -q fastapi nest-asyncio uvicorn pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.

In [3]:

from google.colab import userdata
NGROK = userdata.get('NGROK')

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/drow_ai_backend/drowai_api_backend_small.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

Mounted at /content/drive
/content/drive/MyDrive/drow_ai_backend
/content/drive/MyDrive/drow_ai_backend
/content/drive/MyDrive/drow_ai_backend


In [4]:
#モデルの設定を行う。

#sdxlのモデルはvariant = "fp16"で読み込んでいるため、モデルのsafetensor名にfp16と入ってるモデルを利用してください。
#from_single_file = Trueでsafetensor単体のモデルも読み込めるようになりました。
#その場合は、base_model_pathにsafetensorのpathを指定してください。

config_text = """
[SDXLC]
device = auto
n_steps=4
high_noise_frac=None
seed=42

vae_model_path = None
base_model_path = Asahina2K/Animagine-xl-3.1-diffuser-variant-fp16
refiner_model_path = None

controlnet_path_s = xinsir/controlnet-scribble-sdxl-1.0
control_mode = scribble


lora_weight_path = ./inputs/lcm-animaginexl-3_1.safetensors
lora_weight_path2 = ./inputs/DreamyvibesartstyleSDXL.safetensors
lora_scale = 1.0
lora_scale2 = 1.0
trigger_word = None
trigger_word2 = "Dreamyvibes Artstyle"

select_solver = LCM
;select_solver = DPM
;select_solver = Eulera

use_karras_sigmas = True
scheduler_algorithm_type = dpmsolver++
solver_order = 2

cfg_scale = 2.0
width = 1024
height = 1024
output_type = pil
aesthetic_score = 6
negative_aesthetic_score = 2.5

save_latent_simple = False
save_latent_overstep = False
save_latent_approximation = False

"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

In [5]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from PIL import Image, ImageOps
import io
import base64
import nest_asyncio
from pyngrok import ngrok
from module.module_sdc_small import SDXLC as SDXL
import asyncio
import torch
import numpy as np

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:
app = FastAPI()
sdxl = SDXL()
print("Stable Diffusion XL model loaded successfully")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


Stable Diffusion XL model loaded successfully


In [7]:
# CORSの設定
origins = [
    "*" # フロントエンドのURLを指定
    # 必要に応じて他のオリジンを追加
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],  # すべてのHTTPメソッドを許可
    allow_headers=["*"],  # すべてのヘッダーを許可
)

class ImageData(BaseModel):
    image: str

class InpaintData(BaseModel):
    original_image: str
    mask_image: str

# プロンプトの情報を保持するクラス変数
class PromptData:
    prompt = ""
    negative_prompt = ""

class ApplyPromptsData(BaseModel):
    prompt: str
    negative_prompt: str

class ApplyModeData(BaseModel):
  mode: str

@app.post("/api/process-image")
async def process_image(data: ImageData):
    try:
        # Base64エンコードされた画像データをデコード
        image_data = base64.b64decode(data.image.split(",")[1])
        image = Image.open(io.BytesIO(image_data))

        # 画像の線を赤に変更する処理
        red_image = process_scribble(image)
        print("red_image")

        # 画像をBase64にエンコードして返却
        buffered = io.BytesIO()
        red_image.save(buffered, format="PNG")
        processed_image_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return {"processed_image": f"data:image/png;base64,{processed_image_str}"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


def process_scribble(image: Image.Image) -> Image.Image:

    # ピクセルデータを取得
    datas = image.getdata()

    # 新しいピクセルデータを格納するリスト
    new_data = []

    # ピクセルをループして透明度をチェック
    for item in datas:
        # 完全に透明なピクセル（アルファが0）の場合
        if item[3] == 0:
            # kuro色で置き換え
            new_data.append((0, 0, 0, 255))
        else:
            # そのまま
            new_data.append(item)

    # 新しいピクセルデータで画像を更新
    image.putdata(new_data)

    # 画像をPNG形式で保存
    image.save('inputs/refer.png', 'PNG')
    image = sdxl.generate_image(PromptData.prompt, PromptData.negative_prompt,'inputs/refer.png',controlnet_conditioning_scale = 0.3, mode = "scribble")
    print("image")
    image = image.convert("RGBA")
    return image

@app.post("/api/inpaint")
async def inpaint(data: InpaintData):
    try:
        # Base64エンコードされた元画像とマスク画像をデコード
        original_image_data = base64.b64decode(data.original_image.split(",")[1])
        mask_image_data = base64.b64decode(data.mask_image.split(",")[1])

        original_image = Image.open(io.BytesIO(original_image_data)).convert("RGBA")
        mask_image = Image.open(io.BytesIO(mask_image_data)).convert("L")

        original_image.save('inputs/original_image.png', 'PNG')
        mask_image.save('inputs/mask_image.png', 'PNG')

        # マスクに従って元画像の範囲を消去
        result_image_tensor, mask_tensor = apply_mask(original_image, mask_image)
        #result_image.save('inputs/refer.png', 'PNG')
        image = sdxl.generate_image(PromptData.prompt, PromptData.negative_prompt,result_image_tensor,controlnet_conditioning_scale = 0.9, mode = "inpaint", mask_image = mask_tensor)
        image = image.convert("RGBA")

        # 画像をBase64にエンコードして返却
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        result_image_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return {"inpainted_image": f"data:image/png;base64,{result_image_str}"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

def apply_mask(original_image: Image.Image, mask_image: Image.Image) -> Image.Image:
    # PIL画像をNumPy配列に変換
    original_image = original_image.convert("RGB")
    mask_image = mask_image.convert("L")
    mask_image = mask_image.resize((original_image.size[0],original_image.size[1]), Image.NEAREST)
    original_np = np.array(original_image)
    mask_np = np.array(mask_image)

    #print(original_np.shape)
    
    # NumPy配列をTorchテンソルに変換
    original_tensor = torch.tensor(original_np)
    mask_tensor = torch.tensor(mask_np)
    mask_tensor = mask_tensor.unsqueeze(2)

    masked_tensor = original_tensor.clone()  # 元のテンソルをコピー

    return masked_tensor, mask_tensor


@app.post("/api/apply-prompts")
async def apply_prompts(data: ApplyPromptsData):
    # プロンプトとネガティブプロンプトのデータを保存
    PromptData.prompt = data.prompt
    PromptData.negative_prompt = data.negative_prompt
    print(f"Prompt: {PromptData.prompt}", f"Negative Prompt: {PromptData.negative_prompt}")

    return {"message": "Prompts applied successfully"}


@app.post("/api/change_mode")
async def change_mode(data: ApplyModeData):
  mode = data.mode
  if mode == "inpaint":
    print("loading inpaint mode")
    sdxl.memory_reset_model(mode = "inpaint")
    print("loaded inpaint mode")
  elif mode == "sketch":
    print("loading sketch mode")
    sdxl.memory_reset_model(mode = "sketch")
    print("loaded sketch mode")


In [ ]:
import uvicorn
ngrok.set_auth_token(NGROK)

ngrok_tunnel=ngrok.connect(8000)
print('PUBLIC_URL:',ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app,port=8000)

INFO:     Started server process [2033]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


PUBLIC_URL: https://6f35-34-81-172-75.ngrok-free.app
INFO:     2400:4050:b5a1:9000:65f4:d9df:18c1:21dc:0 - "OPTIONS /api/apply-prompts HTTP/1.1" 200 OK
Prompt: 1 girl ,Yellowish-white hair ,short hair ,red small ribbon,red eyes,red hat ,school uniform ,solo ,smile ,upper body ,Anime ,Japanese,best quality,high quality,ultra highres,ultra quality Negative Prompt: nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]
INFO:     2400:4050:b5a1:9000:65f4:d9df:18c1:21dc:0 - "POST /api/apply-prompts HTTP/1.1" 200 OK
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "OPTIONS /api/process-image HTTP/1.1" 200 OK


  0%|          | 0/4 [00:00<?, ?it/s]

INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/process-image HTTP/1.1" 200 OK
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "OPTIONS /api/change_mode HTTP/1.1" 200 OK
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/change_mode HTTP/1.1" 200 OK
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "OPTIONS /api/apply-prompts HTTP/1.1" 200 OK
Prompt: Cry girl Negative Prompt: 
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/apply-prompts HTTP/1.1" 200 OK
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "OPTIONS /api/inpaint HTTP/1.1" 200 OK


  0%|          | 0/4 [00:00<?, ?it/s]

INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/inpaint HTTP/1.1" 200 OK
Prompt: Black hair Negative Prompt: 
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/apply-prompts HTTP/1.1" 200 OK


  0%|          | 0/4 [00:00<?, ?it/s]

INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/inpaint HTTP/1.1" 200 OK
Prompt: Black hair, cry girl Negative Prompt: 
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/apply-prompts HTTP/1.1" 200 OK


  0%|          | 0/4 [00:00<?, ?it/s]

INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/inpaint HTTP/1.1" 200 OK
INFO:     2400:4050:b5a1:9000:ddb7:10e8:f3e2:d8ac:0 - "POST /api/change_mode HTTP/1.1" 200 OK


1 girl ,Yellowish-white hair ,short hair ,red small ribbon,red eyes,red hat ,school uniform ,solo ,smile ,upper body ,Anime ,Japanese,best quality,high quality,ultra highres,ultra quality


nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]